# Chatterbox TTS - Synthese Vocale Expressive

**Module :** 02-Audio-Advanced  
**Niveau :** Intermediaire  
**Technologies :** Chatterbox Turbo (ResembleAI, MIT license), ~8 GB VRAM  
**Duree estimee :** 45 minutes  

## Objectifs d'Apprentissage

- [ ] Installer et charger le modele Chatterbox TTS
- [ ] Generer de la parole expressive avec controle d'emotion
- [ ] Maitriser les parametres d'exageration et de guidance (cfg_weight)
- [ ] Utiliser le voice conditioning a partir d'un clip audio de reference (6s)
- [ ] Controler la prosodie et l'intonation via les parametres du modele
- [ ] Comparer la qualite avec OpenAI TTS et Kokoro
- [ ] Evaluer les cas d'usage : narration, assistants, e-learning

## Prerequis

- GPU NVIDIA avec au moins 8 GB VRAM
- `pip install chatterbox-tts`
- Notebook 01-5 recommande (pour la comparaison avec Kokoro)

**Navigation :** [<< 01-5](../01-Foundation/01-5-Kokoro-TTS-Local.ipynb) | [Index](../README.md) | [Suivant >>](02-2-XTTS-Voice-Cloning.ipynb)

In [ ]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres Chatterbox
model_name = "chatterbox"           # Modele Chatterbox TTS
device = "cuda"                     # "cuda" ou "cpu"
exaggeration = 0.5                  # Intensite de l'emotion (0.0 a 1.0)
cfg_weight = 0.5                    # Guidance weight (0.0 a 1.0)

# Configuration
generate_audio = True              # Generer les fichiers audio
save_results = True                # Sauvegarder les fichiers generes
compare_emotions = True            # Comparer les differentes emotions
test_voice_conditioning = True     # Tester le voice conditioning

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import time
import gc
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import logging

import numpy as np
import soundfile as sf
from IPython.display import Audio, display, HTML

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.audio_helpers import play_audio, save_audio
        print("Helpers audio importes")
    except ImportError:
        print("Helpers audio non disponibles - mode autonome")

# Repertoires
OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'audio' / 'chatterbox'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('chatterbox_tts')

# Verification GPU
gpu_available = False
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_vram = torch.cuda.get_device_properties(0).total_mem / (1024**3)
        print(f"GPU : {gpu_name} ({gpu_vram:.1f} GB VRAM)")
    else:
        print("GPU non disponible - Chatterbox necessite un GPU pour des performances correctes")
        if device == "cuda":
            device = "cpu"
            print("Fallback vers CPU")
except ImportError:
    print("torch non installe")
    device = "cpu"

print(f"\nChatterbox TTS - Synthese Vocale Expressive")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}, Device : {device}")
print(f"Exaggeration : {exaggeration}, CFG Weight : {cfg_weight}")
print(f"Sortie : {OUTPUT_DIR}")

In [ ]:
# Chargement .env (pour comparaison eventuelle avec OpenAI)
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve - pas de comparaison cloud possible")

openai_key = os.getenv('OPENAI_API_KEY')
if openai_key:
    print(f"OPENAI_API_KEY disponible (pour comparaison)")
else:
    print(f"OPENAI_API_KEY non disponible - comparaison avec OpenAI desactivee")

## Section 1 : Presentation de Chatterbox

Chatterbox est un modele TTS developpe par ResembleAI, sous licence MIT. Sa particularite est le controle fin de l'expressivite vocale : emotion, prosodie, intonation.

### Caracteristiques

| Aspect | Chatterbox Turbo | Kokoro 82M | OpenAI TTS |
|--------|-----------------|-----------|------------|
| Editeur | ResembleAI | Hexgrad | OpenAI |
| Licence | MIT | MIT | Proprietaire |
| Parametres | ~300M | 82M | Non publie |
| VRAM | ~8 GB | ~2 GB | N/A (API) |
| Emotions | Oui (controle fin) | Non | Non |
| Voice conditioning | Oui (6s clip) | Non | Non |
| Qualite (MOS) | ~4.2 | ~4.0 | ~4.5 |
| Cout | Gratuit | Gratuit | $15-30/1M chars |

### Parametres cles

| Parametre | Plage | Description |
|-----------|-------|-------------|
| `exaggeration` | 0.0 - 1.0 | Intensite de l'expressivite (0 = neutre, 1 = tres expressif) |
| `cfg_weight` | 0.0 - 1.0 | Guidance : fidelite au conditionnement (0 = libre, 1 = strict) |

In [ ]:
# Chargement du modele Chatterbox
print("CHARGEMENT DU MODELE CHATTERBOX")
print("=" * 45)

chatterbox_loaded = False

try:
    from chatterbox.tts import ChatterboxTTS

    print(f"Chargement Chatterbox sur {device}...")
    start_time = time.time()
    model = ChatterboxTTS.from_pretrained(device=device)
    load_time = time.time() - start_time
    chatterbox_loaded = True

    print(f"Modele charge en {load_time:.1f}s")
    print(f"Device : {device}")
    print(f"Sample rate : {model.sr} Hz")

    if gpu_available:
        vram_used = torch.cuda.memory_allocated(0) / (1024**3)
        print(f"VRAM utilisee : {vram_used:.2f} GB")

except ImportError:
    print("chatterbox-tts non installe")
    print("Installation : pip install chatterbox-tts")
except Exception as e:
    print(f"Erreur lors du chargement : {type(e).__name__} - {str(e)[:200]}")

## Section 2 : Premiere generation

La generation avec Chatterbox utilise la methode `model.generate()`. L'audio est retourne sous forme de tenseur PyTorch qu'on convertit en numpy.

| Parametre | Type | Description |
|-----------|------|-------------|
| `text` | str | Texte a synthetiser |
| `audio_prompt_path` | str/None | Chemin vers un clip audio de reference (optionnel) |
| `exaggeration` | float | Controle d'expressivite (0.0-1.0) |
| `cfg_weight` | float | Guidance weight (0.0-1.0) |

In [ ]:
# Premiere generation TTS avec Chatterbox
print("PREMIERE GENERATION CHATTERBOX")
print("=" * 45)

sample_text = (
    "Welcome to this tutorial on expressive speech synthesis. "
    "Chatterbox allows you to control the emotion and style of generated speech "
    "with remarkable precision."
)

print(f"Texte : {sample_text}")
print(f"Exaggeration : {exaggeration}")
print(f"CFG Weight : {cfg_weight}")

if chatterbox_loaded and generate_audio:
    start_time = time.time()

    wav = model.generate(
        text=sample_text,
        exaggeration=exaggeration,
        cfg_weight=cfg_weight
    )

    gen_time = time.time() - start_time

    # Conversion tenseur -> numpy
    if hasattr(wav, 'cpu'):
        samples = wav.cpu().numpy().squeeze()
    else:
        samples = np.array(wav).squeeze()

    sample_rate = model.sr
    duration = len(samples) / sample_rate

    print(f"\nGeneration reussie")
    print(f"  Duree audio : {duration:.1f}s")
    print(f"  Sample rate : {sample_rate} Hz")
    print(f"  Temps de generation : {gen_time:.2f}s")
    print(f"  Ratio temps reel : {duration / gen_time:.1f}x")

    # Ecoute
    print(f"\nEcoute :")
    display(Audio(data=samples, rate=sample_rate))

    # Sauvegarde
    if save_results:
        filepath = OUTPUT_DIR / f"chatterbox_basic.wav"
        sf.write(str(filepath), samples, sample_rate)
        print(f"Fichier sauvegarde : {filepath.name}")
else:
    print("Modele non charge ou generation desactivee")

### Interpretation : Premiere generation

| Aspect | Valeur typique | Signification |
|--------|---------------|---------------|
| Ratio temps reel | 3-10x (GPU) | Chatterbox est plus lent que Kokoro mais reste temps reel |
| Sample rate | 24000 Hz | Standard pour la parole haute qualite |
| VRAM | ~8 GB | Necessaire pour le modele complet |

> **Note technique** : Chatterbox genere nativement en anglais. Le support d'autres langues est experimental et peut produire des resultats variables.

## Section 3 : Controle des emotions

L'atout principal de Chatterbox est le controle fin de l'expressivite via le parametre `exaggeration` :

| Exaggeration | Resultat | Cas d'usage |
|-------------|----------|-------------|
| 0.0 | Voix neutre, monotone | Lecture factuelle, annonces |
| 0.25 | Legerement expressif | Narration documentaire |
| 0.5 | Expressivite moderee | E-learning, podcasts |
| 0.75 | Tres expressif | Histoires, contenus engageants |
| 1.0 | Exagere | Personnages, animations |

In [ ]:
# Test des differents niveaux d'expressivite
print("CONTROLE DES EMOTIONS")
print("=" * 45)

emotion_text = (
    "I am absolutely thrilled to share this incredible news with you today. "
    "This is going to change everything."
)

exaggeration_levels = [0.0, 0.25, 0.5, 0.75, 1.0]
emotion_results = {}

if chatterbox_loaded and generate_audio and compare_emotions:
    for exag in exaggeration_levels:
        print(f"\nExaggeration = {exag}")
        start_time = time.time()

        wav = model.generate(
            text=emotion_text,
            exaggeration=exag,
            cfg_weight=cfg_weight
        )

        gen_time = time.time() - start_time

        if hasattr(wav, 'cpu'):
            samples = wav.cpu().numpy().squeeze()
        else:
            samples = np.array(wav).squeeze()

        sample_rate = model.sr
        duration = len(samples) / sample_rate

        emotion_results[exag] = {
            "duration": duration,
            "gen_time": gen_time,
            "samples": samples,
            "sample_rate": sample_rate
        }

        print(f"  Duree : {duration:.1f}s | Temps : {gen_time:.2f}s")
        display(Audio(data=samples, rate=sample_rate))

        if save_results:
            filepath = OUTPUT_DIR / f"emotion_exag_{exag:.2f}.wav"
            sf.write(str(filepath), samples, sample_rate)

    # Tableau recapitulatif
    print(f"\nRecapitulatif des generations :")
    print(f"{'Exaggeration':<15} {'Duree (s)':<12} {'Temps gen (s)':<15}")
    print("-" * 42)
    for exag, data in emotion_results.items():
        print(f"{exag:<15.2f} {data['duration']:<12.1f} {data['gen_time']:<15.2f}")
else:
    print("Test des emotions desactive ou modele non charge")

### Interpretation : Controle des emotions

| Exaggeration | Observation | Recommandation |
|-------------|-------------|----------------|
| 0.0 | Voix plate, robotique | Eviter sauf pour du contenu tres factuel |
| 0.25-0.5 | Naturel, professionnel | Ideal pour la majorite des cas d'usage |
| 0.75 | Tres expressif, engageant | Bon pour le storytelling |
| 1.0 | Parfois exagere | A utiliser avec precaution |

**Points cles** :
1. L'exaggeration n'affecte pas significativement le temps de generation
2. La duree audio peut varier selon l'expressivite (pauses, debit)
3. La valeur 0.5 est un bon point de depart pour la plupart des usages

## Section 4 : Voice conditioning

Chatterbox peut reproduire le timbre d'une voix de reference a partir d'un clip audio de ~6 secondes. C'est la fonctionnalite de **voice conditioning** (ou voice prompting).

### Principe

| Etape | Description |
|-------|-------------|
| 1. Clip de reference | Audio WAV de ~6s avec la voix cible |
| 2. Extraction | Le modele extrait les caracteristiques vocales |
| 3. Generation | Le texte est synthetise avec le timbre de reference |

> **Note ethique** : Le clonage vocal souleve des questions importantes. Utilisez uniquement des voix pour lesquelles vous avez le consentement explicite du locuteur.

In [ ]:
# Voice conditioning avec clip de reference
print("VOICE CONDITIONING")
print("=" * 45)

conditioning_text = (
    "This speech is generated using voice conditioning. "
    "The model captures the timbre and characteristics of the reference speaker."
)

if chatterbox_loaded and generate_audio and test_voice_conditioning:
    # Creer un clip de reference synthetique pour la demonstration
    # En production, on utiliserait un vrai enregistrement vocal
    print("Creation d'un clip de reference synthetique...")

    ref_wav = model.generate(
        text="Hello, this is my reference voice clip for conditioning.",
        exaggeration=0.3,
        cfg_weight=0.5
    )

    if hasattr(ref_wav, 'cpu'):
        ref_samples = ref_wav.cpu().numpy().squeeze()
    else:
        ref_samples = np.array(ref_wav).squeeze()

    # Sauvegarder le clip de reference
    ref_path = OUTPUT_DIR / "reference_clip.wav"
    sf.write(str(ref_path), ref_samples, model.sr)
    print(f"Clip de reference : {ref_path.name} ({len(ref_samples)/model.sr:.1f}s)")
    display(Audio(data=ref_samples, rate=model.sr))

    # Generation avec voice conditioning
    print(f"\nGeneration avec voice conditioning...")
    start_time = time.time()

    wav_conditioned = model.generate(
        text=conditioning_text,
        audio_prompt_path=str(ref_path),
        exaggeration=exaggeration,
        cfg_weight=cfg_weight
    )

    gen_time = time.time() - start_time

    if hasattr(wav_conditioned, 'cpu'):
        cond_samples = wav_conditioned.cpu().numpy().squeeze()
    else:
        cond_samples = np.array(wav_conditioned).squeeze()

    duration = len(cond_samples) / model.sr

    print(f"  Duree : {duration:.1f}s | Temps : {gen_time:.2f}s")
    display(Audio(data=cond_samples, rate=model.sr))

    # Generation sans conditioning pour comparaison
    print(f"\nGeneration SANS conditioning (meme texte) :")
    wav_uncond = model.generate(
        text=conditioning_text,
        exaggeration=exaggeration,
        cfg_weight=cfg_weight
    )

    if hasattr(wav_uncond, 'cpu'):
        uncond_samples = wav_uncond.cpu().numpy().squeeze()
    else:
        uncond_samples = np.array(wav_uncond).squeeze()

    print(f"  Duree : {len(uncond_samples)/model.sr:.1f}s")
    display(Audio(data=uncond_samples, rate=model.sr))

    if save_results:
        sf.write(str(OUTPUT_DIR / "conditioned.wav"), cond_samples, model.sr)
        sf.write(str(OUTPUT_DIR / "unconditioned.wav"), uncond_samples, model.sr)
        print(f"Fichiers sauvegardes")
else:
    print("Voice conditioning desactive ou modele non charge")

### Interpretation : Voice conditioning

| Aspect | Avec conditioning | Sans conditioning |
|--------|------------------|-------------------|
| Timbre | Proche de la reference | Voix par defaut du modele |
| Latence | Legerement plus elevee | Standard |
| Qualite | Tres bonne si clip de qualite | Constante |

**Points cles** :
1. Le clip de reference doit etre de bonne qualite (pas de bruit de fond)
2. Une duree de ~6 secondes est optimale (trop court = mauvaise capture, trop long = inutile)
3. Le voice conditioning fonctionne mieux avec un cfg_weight plus eleve (0.5-0.8)

> **Note ethique** : Ne clonez jamais une voix sans le consentement explicite de la personne concernee.

In [ ]:
# Mode interactif - Generation personnalisee
if notebook_mode == "interactive" and not skip_widgets:
    print("MODE INTERACTIF - GENERATION PERSONNALISEE")
    print("=" * 50)
    print("\nEntrez un texte a synthetiser avec Chatterbox :")
    print("(Laissez vide pour passer a la suite)")

    try:
        user_text = input("\nVotre texte : ")

        if user_text.strip() and chatterbox_loaded:
            user_exag = input(f"Exaggeration [{exaggeration}] (0.0-1.0) : ").strip()
            user_exag = float(user_exag) if user_exag else exaggeration

            user_cfg = input(f"CFG Weight [{cfg_weight}] (0.0-1.0) : ").strip()
            user_cfg = float(user_cfg) if user_cfg else cfg_weight

            print(f"\nGeneration en cours (exag={user_exag}, cfg={user_cfg})...")
            start_time = time.time()

            wav = model.generate(
                text=user_text,
                exaggeration=user_exag,
                cfg_weight=user_cfg
            )

            gen_time = time.time() - start_time

            if hasattr(wav, 'cpu'):
                samples = wav.cpu().numpy().squeeze()
            else:
                samples = np.array(wav).squeeze()

            print(f"Duree : {len(samples)/model.sr:.1f}s | Temps : {gen_time:.2f}s")
            display(Audio(data=samples, rate=model.sr))

            if save_results:
                ts = datetime.now().strftime('%Y%m%d_%H%M%S')
                filepath = OUTPUT_DIR / f"custom_{ts}.wav"
                sf.write(str(filepath), samples, model.sr)
                print(f"Sauvegarde : {filepath.name}")
        else:
            print("Mode interactif ignore")

    except (KeyboardInterrupt, EOFError):
        print("Mode interactif interrompu")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("Mode interactif non disponible (execution automatisee)")
        else:
            print(f"Erreur : {error_type} - {str(e)[:100]}")
else:
    print("Mode batch - Interface interactive desactivee")

## Bonnes pratiques et guide de decision

### Quand utiliser Chatterbox

| Scenario | Recommandation | Raison |
|----------|---------------|--------|
| Narration expressive | Chatterbox | Controle fin des emotions |
| Voice conditioning | Chatterbox | Fonctionnalite unique |
| TTS basique rapide | Kokoro | Plus leger, plus rapide |
| Qualite maximale | OpenAI TTS-HD | Meilleure naturalite |
| Multilangue | XTTS v2 | 17 langues supportees |

### Optimisation des parametres

| Usage | Exaggeration | CFG Weight |
|-------|-------------|------------|
| Narration documentaire | 0.2-0.3 | 0.5 |
| E-learning | 0.4-0.5 | 0.5 |
| Storytelling | 0.6-0.8 | 0.4 |
| Personnages animes | 0.8-1.0 | 0.3 |

In [ ]:
# Statistiques de session et prochaines etapes
print("STATISTIQUES DE SESSION")
print("=" * 45)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Modele : {model_name}")
print(f"Device : {device}")
print(f"Exaggeration : {exaggeration}")
print(f"CFG Weight : {cfg_weight}")
print(f"Modele charge : {'Oui' if chatterbox_loaded else 'Non'}")

if gpu_available:
    vram_current = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"VRAM utilisee : {vram_current:.2f} GB")

if save_results:
    saved = list(OUTPUT_DIR.glob('*'))
    total_size = sum(f.stat().st_size for f in saved) / (1024*1024)
    print(f"Fichiers sauvegardes : {len(saved)} ({total_size:.1f} MB) dans {OUTPUT_DIR}")

# Liberation memoire
if chatterbox_loaded:
    print(f"\nLiberation du modele...")
    del model
    gc.collect()
    if gpu_available:
        torch.cuda.empty_cache()
    print(f"Memoire liberee")

print(f"\nPROCHAINES ETAPES")
print(f"1. Decouvrir le voice cloning avec XTTS v2 (02-2)")
print(f"2. Explorer la generation musicale avec MusicGen (02-3)")
print(f"3. Tester la separation de sources avec Demucs (02-4)")
print(f"4. Comparer tous les modeles audio (03-1)")

print(f"\nNotebook Chatterbox TTS termine - {datetime.now().strftime('%H:%M:%S')}")